# 1.tf.keras.losses 作用
`tf.keras.losses` 模块主要用于定义 损失函数（`Loss Function`），它的作用是衡量模型预测值 (y_pred) 与真实值 (y_true) 之间的差异，并为优化器（`Optimizer`）提供梯度信息，从而指导权重更新。

---
## 2. 常见损失函数分类
| 类别        | 常用函数                                                       | 典型场景              |
| --------- | ---------------------------------------------------------- | ----------------- |
| **回归损失**  | `MeanSquaredError`, `MeanAbsoluteError`, `Huber`           | 预测连续值             |
| **二分类损失** | `BinaryCrossentropy`                                       | 二分类（sigmoid 输出）   |
| **多分类损失** | `CategoricalCrossentropy`, `SparseCategoricalCrossentropy` | 多类别分类（softmax 输出） |
| **对比损失**  | `CosineSimilarity`, `KLDivergence`                         | 语义相似度、分布差异        |
| **自定义损失** | 继承 `tf.keras.losses.Loss`                                  | 特殊任务或业务需求         |
## 3. 使用方式
loss 可以有三种使用形式：

### ① 直接用字符串

In [ ]:
model.compile(loss='mse', optimizer='adam')


### ② 使用类（推荐）

In [ ]:
from tensorflow.keras.losses import MeanSquaredError
model.compile(loss=MeanSquaredError(), optimizer='adam')


### ③ 自定义函数

In [ ]:
def my_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

model.compile(loss=my_loss, optimizer='adam')


## 4. 参数讲解（以 BinaryCrossentropy 为例）

In [ ]:
tf.keras.losses.BinaryCrossentropy(
    from_logits=False,
    label_smoothing=0.0,
    axis=-1,
    reduction=tf.keras.losses.Reduction.AUTO,
    name='binary_crossentropy'
)


| 参数                   | 类型      | 说明                                                         |
| -------------------- | ------- | ---------------------------------------------------------- |
| **from\_logits**     | `bool`  | 如果模型输出**未经过 Sigmoid**，设为 `True`（内部会自动加 sigmoid），否则 `False` |
| **label\_smoothing** | `float` | 标签平滑系数，例如 0.1 会把 1 变为 0.9，0 变为 0.1                         |
| **axis**             | `int`   | 计算损失的维度，默认 `-1`                                            |
| **reduction**        | 枚举值     | 如何聚合损失值：`SUM`, `SUM_OVER_BATCH_SIZE`, `NONE`, `AUTO`       |
| **name**             | `str`   | 损失函数名字，用于日志显示                                              |


## 5. loss 与 metrics 的区别
| 对比项           | Loss        | Metrics         |
| ------------- | ----------- | --------------- |
| **作用**        | 优化目标，参与反向传播 | 性能评估，不参与反向传播    |
| **计算时机**      | 每个 batch    | 每个 batch（累计求平均） |
| **是否影响梯度**    | ✅           | ❌               |
| **是否在优化器中使用** | ✅           | ❌               |


## 6. 最小示例

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# 简单二分类模型
model = models.Sequential([
    layers.Dense(8, activation='relu', input_shape=(4,)),
    layers.Dense(1, activation='sigmoid')
])

# 编译（使用损失函数）
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=['accuracy']
)

# 生成假数据
import numpy as np
X = np.random.random((10, 4))
y = np.random.randint(0, 2, size=(10, 1))

# 训练
model.fit(X, y, epochs=2)


这里：

loss 是 BinaryCrossentropy（指导训练）

metrics 是 accuracy（仅显示性能，不参与训练）

# 📌 TensorFlow tf.keras.losses 速查表
| 损失函数                                       | 适用任务            | 主要参数                             | 备注 / 注意事项                                     |
| ------------------------------------------ | --------------- | -------------------------------- | --------------------------------------------- |
| **MeanSquaredError** (`'mse'`)             | 回归              | `reduction`, `name`              | 均方误差，预测连续值时常用                                 |
| **MeanAbsoluteError** (`'mae'`)            | 回归              | 同上                               | 对异常值不敏感，适合稳健回归                                |
| **MeanAbsolutePercentageError** (`'mape'`) | 回归              | 同上                               | 预测误差的百分比，适合关注相对误差的场景                          |
| **Huber**                                  | 回归              | `delta`（切换 MAE 与 MSE 的阈值）        | 结合 MSE 和 MAE，抗噪性好                             |
| **BinaryCrossentropy**                     | 二分类             | `from_logits`, `label_smoothing` | Sigmoid 输出设 `from_logits=False`，无激活输出设 `True` |
| **CategoricalCrossentropy**                | 多分类（one-hot 标签） | `from_logits`, `label_smoothing` | Softmax 输出设 `False`，无激活设 `True`               |
| **SparseCategoricalCrossentropy**          | 多分类（整数标签）       | 同上                               | 输入标签形如 `[0, 2, 1]` 而非 one-hot                 |
| **KLDivergence**                           | 概率分布差异          | `reduction`, `name`              | 用于分布匹配任务，如知识蒸馏                                |
| **CosineSimilarity**                       | 相似度             | `axis`（计算维度）                     | 用于向量相似度任务（值越大越相似）                             |
| **Poisson**                                | 计数预测            | 同上                               | 适合预测泊松分布数据（如事件发生次数）                           |
| **LogCosh**                                | 回归              | 同上                               | 对异常值更稳健，类似 Huber                              |
| **Hinge**                                  | 二分类（SVM 风格）     | `reduction`, `name`              | 标签需为 {-1, 1}                                  |
| **SquaredHinge**                           | 二分类（SVM 风格）     | 同上                               | Hinge 的平方版本                                   |
| **CategoricalHinge**                       | 多分类（SVM 风格）     | 同上                               | 标签需为 one-hot                                  |

# 📍 参数解释（通用）
| 参数                   | 类型      | 作用                                           |
| -------------------- | ------- | -------------------------------------------- |
| **from\_logits**     | `bool`  | 是否输入未经过激活函数（如 Sigmoid/Softmax）               |
| **label\_smoothing** | `float` | 标签平滑，减少过拟合（0\~1）                             |
| **delta**            | `float` | Huber/LogCosh 的切换阈值                          |
| **axis**             | `int`   | CosineSimilarity 的计算维度                       |
| **reduction**        | 枚举值     | `SUM`, `SUM_OVER_BATCH_SIZE`, `NONE`, `AUTO` |
| **name**             | `str`   | 损失函数的名称                                      |

# 📚 快速选择建议
回归 → MSE, MAE, Huber

二分类 → BinaryCrossentropy

多分类（one-hot） → CategoricalCrossentropy

多分类（整数标签） → SparseCategoricalCrossentropy

分布匹配 → KLDivergence

相似度任务 → CosineSimilarity

SVM 风格 → Hinge / SquaredHinge / CategoricalHinge